<a href="https://colab.research.google.com/github/janaghoniem/Housing-Prices-Prediction-using-Machine-Learning/blob/main/Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/cleaned_dataset.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   float64
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

# Feature Selection

## 1. PCA/LDA + Recursive Feature Elimination
PCA is used to reduce the dimensionality of the dataset while retaining most of the variance in the data.
Recursive Feature Elimination (RFE) to select the most important features. (wrapper method)

In [8]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/scaled_dataset.csv')

# Separate features and target
X = data.drop('SalePrice', axis=1)  # Features (all columns except 'SalePrice')
y = data['SalePrice']  # Target variable

# Step 1: Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_pca = pca.fit_transform(X)  # Apply PCA to the already scaled data

print(f"Original number of features: {X.shape[1]}")
print(f"Reduced number of features after PCA: {X_pca.shape[1]}")

# Step 2: Apply Recursive Feature Elimination (RFE)
model = LinearRegression()  # Initialize Linear Regression model
rfe = RFE(estimator=model, n_features_to_select=5)  # Select top 5 features
X_rfe = rfe.fit_transform(X_pca, y)  # Fit RFE and transform the PCA-transformed data

# Get the selected features
selected_features = rfe.support_  # Boolean mask of selected features
print(f"Selected features mask: {selected_features}")

# Step 3: Train and evaluate the model
X_train, X_test, y_train, y_test = train_test_split(X_rfe, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)  # Train the Linear Regression model
y_pred = model.predict(X_test)  # Make predictions

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Original number of features: 168
Reduced number of features after PCA: 16
Selected features mask: [False  True False False False  True False  True False False  True False
 False  True False False]
Mean Squared Error: 0.17922204116117624


In [4]:
# Check unique categories in each categorical column
for col in categorical_cols:
    print(f"{col}: {X[col].nunique()} unique categories")

NameError: name 'categorical_cols' is not defined

## 2. RFE with bayesian ridge

In [6]:
import pandas as pd
from sklearn.linear_model import BayesianRidge
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/scaled_dataset.csv')

# Separate features and target
X = data.drop('SalePrice', axis=1)  # Features (all columns except 'SalePrice')
y = data['SalePrice']  # Target variable

# Step 1: Apply Recursive Feature Elimination (RFE) with Bayesian Ridge
model = BayesianRidge()  # Initialize Bayesian Ridge model
rfe = RFE(estimator=model, n_features_to_select=5)  # Select top 5 features
X_rfe = rfe.fit_transform(X, y)  # Fit RFE and transform the data

# Get the selected features
selected_features = X.columns[rfe.support_]  # Get the names of the selected features
print("Selected features:", selected_features)

# Step 2: Train and evaluate the model
X_train, X_test, y_train, y_test = train_test_split(X_rfe, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)  # Train the Bayesian Ridge model
y_pred = model.predict(X_test)  # Make predictions

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Selected features: Index(['Condition2_PosN', 'RoofMatl_CompShg', 'RoofMatl_Roll',
       'RoofMatl_WdShake', 'RoofMatl_WdShngl'],
      dtype='object')
Mean Squared Error: 0.7315471857821483


In [12]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import BayesianRidge
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/cleaned_dataset.csv')

# Separate features and target
X = data.drop('SalePrice', axis=1)  # Features (all columns except 'SalePrice')
y = data['SalePrice']  # Target variable

# Identify categorical and numeric columns
categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Create a ColumnTransformer to preprocess the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),  # Standardize numeric columns
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)  # One-hot encode categorical columns
    ]
)

# Step 1: Preprocess the data (standardize numeric and encode categorical)
X_preprocessed = preprocessor.fit_transform(X)

# Get feature names after preprocessing
numeric_feature_names = numeric_cols.tolist()
categorical_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
all_feature_names = numeric_feature_names + categorical_feature_names.tolist()

# Step 2: Initialize Bayesian Ridge model
model = BayesianRidge()

# Step 3: Apply Recursive Feature Elimination (RFE)
rfe = RFE(estimator=model, n_features_to_select=5)  # Select top 5 features
X_rfe = rfe.fit_transform(X_preprocessed, y)

# Get the selected features
selected_features = rfe.support_
selected_feature_names = [all_feature_names[i] for i in range(len(all_feature_names)) if selected_features[i]]

print("Selected features:", selected_feature_names)
print("Feature ranking:", rfe.ranking_)

# Step 4: Train and evaluate the model
X_train, X_test, y_train, y_test = train_test_split(X_rfe, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Selected features: ['MSZoning_C (all)', 'Neighborhood_NoRidge', 'Exterior1st_BrkComm', 'KitchenQual_Ex', 'Functional_Maj2']
Feature ranking: [202 242  37  25  50  21 125 272 249 277  77  44  71 234 225   9 170 268
 173 179 246 141 146 138 273  61 157 189 232 276 275 274 235 260 254 222
   1  42  59  60  98  64  91 215 130  68 209  85 134  86 135  57  83 147
 109  65  62 216 230  81  36 266 186 188  35 199 156  17  54 154 111   2
  79  87 150  84   1   5  90 224  88 262  75   4 155 194 100 184  58  95
  63  33 160 196  55 253  27  28  12   8 178 192  22 248 201  10 110 200
 103  51  70  94 211 101  34  69 265 180  92 181  72  24   6 206  13  16
 226 140 113  14 172 219   1  45 102 208 116  76 151 236 149 258  97 107
 175 238 163 142 182  66 176 240 106 251 195 270 204 210  96 108 239 247
 237 221 220  52  89 148 187 231 112 191 161 256  20  23  47 267 123 122
 212 120 245  48 244 190 217  11 205 131 183 115 213 114 133 159 158 218
 136  80 185 264 119 198 197 174 139  18  15  99  30  67

## 3. Recursive Feature Elimination with Cross-Validation (RFECV)

In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/cleaned_dataset.csv')

# Separate features and target
X = data.drop('SalePrice', axis=1)  # Features (all columns except 'SalePrice')
y = data['SalePrice']  # Target variable

# Identify categorical and numeric columns
categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Create a ColumnTransformer to preprocess the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),  # Standardize numeric columns
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)  # One-hot encode categorical columns
    ]
)

# Step 1: Preprocess the data (standardize numeric and encode categorical)
X_preprocessed = preprocessor.fit_transform(X)

# Get feature names after preprocessing
numeric_feature_names = numeric_cols.tolist()
categorical_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
all_feature_names = numeric_feature_names + categorical_feature_names.tolist()

# Convert preprocessed data back to a DataFrame (for easier handling)
X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=all_feature_names)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed_df, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestRegressor(random_state=42)

# Apply RFECV
selector = RFECV(model, step=1, cv=5)  # Use 5-fold cross-validation
selector = selector.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[selector.support_]
print("Selected features (RFECV):", selected_features)

# Train and evaluate the model
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (RFECV): {mse}")

Selected features (RFECV): Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'MoSold', 'YrSold', 'MSZoning_C (all)', 'MSZoning_RL',
       'MSZoning_RM', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_Lvl',
       'LandSlope_Gtl', 'Neighborhood_Crawfor', 'Neighborhood_Edwards',
       'Neighborhood_OldTown', 'Condition1_Norm', 'HouseStyle_1.5Fin',
       'RoofStyle_Gable', 'Exterior1st_MetalSd', 'Exterior2nd_Brk Cmn',
       'Exterior2nd_VinylSd', 'ExterQual_Ex', 'ExterQual_Gd', 'ExterQual_TA',
       'ExterCond_TA', 'Foundation_PConc', 'BsmtQual_Ex', 'BsmtQual_Gd',
       'BsmtCond_Fa', 'BsmtCond_TA', 'BsmtExposure_Gd', 'BsmtExposur

## 4. LASSO with Filter Preprocessing

# EL BLOCK DAH YA HAYAAAAA :

In [17]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/scaled_dataset.csv')

# Separate features and target
X = data.drop('SalePrice', axis=1)  # Features (all columns except 'SalePrice')
y = data['SalePrice']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 1: Apply LASSO for feature selection
lasso = Lasso(alpha=0.01)  # Adjust alpha for regularization strength
selector = SelectFromModel(lasso)  # Use LASSO to select features
selector.fit(X_train, y_train)  # Fit LASSO on the training data

# Get the selected features
selected_features = X_train.columns[selector.get_support()]  # Get the names of the selected features
print("Selected features (LASSO):", selected_features)

# Step 2: Train and evaluate the model
X_train_selected = selector.transform(X_train)  # Transform training data to selected features
X_test_selected = selector.transform(X_test)  # Transform testing data to selected features

# Train a RandomForestRegressor on the selected features
model = RandomForestRegressor(random_state=42)
model.fit(X_train_selected, y_train)  # Train the model
y_pred = model.predict(X_test_selected)  # Make predictions

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (LASSO): {mse}")


selected_data = data[selected_features.tolist() + ['SalePrice']]

# Save the new DataFrame to a CSV file
selected_data.to_csv('/content/drive/MyDrive/Machine Learning/selected_features_dataset_p.csv', index=False)
print("Selected features and target saved to 'selected_features_dataset.csv'.")



Selected features (LASSO): Index(['MSSubClass', 'LotArea', 'OverallCond', 'YearRemodAdd', 'BsmtQual',
       'BsmtExposure', '1stFlrSF', '2ndFlrSF', 'TotRmsAbvGrd', 'Fireplaces',
       'FireplaceQu', 'GarageYrBlt', 'GarageCars', 'WoodDeckSF', 'TotalBath',
       'TotalPorchSF', 'HouseAge', 'TotalSqFt', 'TotalBsmtFinScore',
       'NeighborhoodQuality', 'KitchenQual_ordinal', 'HeatingQC_ordinal',
       'MSZoning_RM', 'Condition1_Norm', 'Foundation_PConc',
       'GarageFinish_Unf'],
      dtype='object')
Mean Squared Error (LASSO): 0.08259583352954884
Selected features and target saved to 'selected_features_dataset.csv'.
Mean Squared Error (Linear Regression): 0.08456310113841625
R² Score (Linear Regression): 0.887331604902248


In [12]:
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/scaled_dataset.csv')

# Separate features and target
X = data.drop('SalePrice', axis=1)  # Features (all columns except 'SalePrice')
y = data['SalePrice']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 1: Hyperparameter Tuning for LASSO (Feature Selection)
# Define the parameter grid for LASSO
lasso_param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10]  # Regularization strength
}

# Initialize GridSearchCV for LASSO
lasso = Lasso()
lasso_grid_search = GridSearchCV(estimator=lasso, param_grid=lasso_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
lasso_grid_search.fit(X_train, y_train)

# Get the best LASSO model
best_lasso = lasso_grid_search.best_estimator_
print("Best LASSO parameters:", lasso_grid_search.best_params_)

# Step 2: Apply LASSO for feature selection using the best model
selector = SelectFromModel(best_lasso)
selector.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[selector.get_support()]
print("Selected features (LASSO):", selected_features)

# Step 3: Transform the data to include only the selected features
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Step 4: Hyperparameter Tuning for Linear Regression
# Define the parameter grid for Linear Regression
linear_param_grid = {
    'fit_intercept': [True, False],  # Whether to calculate the intercept
    'positive': [True, False]  # Whether to force coefficients to be positive
}

# Initialize GridSearchCV for Linear Regression
linear_model = LinearRegression()
linear_grid_search = GridSearchCV(estimator=linear_model, param_grid=linear_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
linear_grid_search.fit(X_train_selected, y_train)

# Get the best Linear Regression model
best_linear_model = linear_grid_search.best_estimator_
print("Best Linear Regression parameters:", linear_grid_search.best_params_)

# Step 5: Train the best Linear Regression model on the selected features
best_linear_model.fit(X_train_selected, y_train)

# Step 6: Make predictions
y_pred = best_linear_model.predict(X_test_selected)

# Step 7: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (Linear Regression): {mse}")
print(f"R² Score (Linear Regression): {r2}")

Best LASSO parameters: {'alpha': 0.001}
Selected features (LASSO): Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallCond', 'YearRemodAdd',
       'MasVnrArea', 'BsmtQual', 'BsmtExposure', 'BsmtUnfSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'BedroomAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'FireplaceQu', 'GarageYrBlt', 'GarageCars', 'PavedDrive',
       'WoodDeckSF', 'MoSold', 'YrSold', 'TotalBath', 'TotalPorchSF',
       'HouseAge', 'IsRemodeled', 'TotalSqFt', 'TotalBsmtFinScore',
       'NeighborhoodQuality', 'ExterQual_ordinal', 'KitchenQual_ordinal',
       'HeatingQC_ordinal', 'MSZoning_FV', 'MSZoning_RM', 'LotShape_IR2',
       'LotShape_IR3', 'LandContour_HLS', 'LandContour_Lvl',
       'LotConfig_CulDSac', 'LotConfig_FR2', 'LandSlope_Mod',
       'Condition1_Norm', 'Condition1_RRAe', 'Condition1_RRAn',
       'Condition2_PosN', 'BldgType_Duplex', 'BldgType_TwnhsE',
       'HouseStyle_2Story', 'RoofStyle_Gable', 'Exterior1st_BrkFace',
       'Exterior1st_HdBoard'

In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/scaled_dataset.csv')

# Separate features and target
X = data.drop('SalePrice', axis=1)  # Features (all columns except 'SalePrice')
y = data['SalePrice']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 1: Hyperparameter Tuning for RandomForestRegressor (Feature Selection)
# Define the parameter grid for RandomForestRegressor
rf_param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10]  # Minimum number of samples required to split a node
}

# Initialize GridSearchCV for RandomForestRegressor
rf = RandomForestRegressor(random_state=42)
rf_grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
rf_grid_search.fit(X_train, y_train)

# Get the best RandomForestRegressor model
best_rf = rf_grid_search.best_estimator_
print("Best RandomForestRegressor parameters:", rf_grid_search.best_params_)

# Step 2: Apply RandomForestRegressor for feature selection using the best model
selector = SelectFromModel(best_rf, threshold='median')  # Use median importance as the threshold
selector.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[selector.get_support()]
print("Selected features (RandomForestRegressor):", selected_features)

# Step 3: Transform the data to include only the selected features
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Step 4: Hyperparameter Tuning for Linear Regression
# Define the parameter grid for Linear Regression
linear_param_grid = {
    'fit_intercept': [True, False],  # Whether to calculate the intercept
    'positive': [True, False]  # Whether to force coefficients to be positive
}

# Initialize GridSearchCV for Linear Regression
linear_model = LinearRegression()
linear_grid_search = GridSearchCV(estimator=linear_model, param_grid=linear_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
linear_grid_search.fit(X_train_selected, y_train)

# Get the best Linear Regression model
best_linear_model =h.best_estimator_
print("Best Linear Regression parameters:", linear_grid_search.best_params_)

# Step 5: Train the best Linear Regression model on the selected features
best_linear_model.fit(X_train_selected, y_train)

# Step 6: Make predictions
y_pred = best_linear_model.predict(X_test_selected)

# Step 7: Evaluat linear_grid_searce the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (Linear Regression): {mse}")
print(f"R² Score (Linear Regression): {r2}")

Best RandomForestRegressor parameters: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 300}
Selected features (RandomForestRegressor): Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterCond', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'BedroomAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'FireplaceQu', 'GarageYrBlt',
       'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF',
       'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'TotalBath', 'TotalPorchSF',
       'HouseAge', 'IsRemodeled', 'TotalSqFt', 'HasGarage', 'QualitySize',
       'TotalBsmtFinScore', 'BsmtFinRatio', 'NeighborhoodQuality',
       'ExterQual_ordinal', 'KitchenQual_ordinal', 'HeatingQC_ordinal',
       'MSZoning_RL', 'MSZoning_RM', 'LotShape_Reg', 'LandContour_Lvl',
       'LotConfig_CulDSac', 'LotConfig_Inside', 'LandSlope_Mod',
       'Condition1_Norm', 'HouseStyle_1Story', 'HouseSty

## 5. Boruta with Random Forest

In [ ]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
model = RandomForestRegressor(n_estimators=100, max_depth=5)

# Apply Boruta
boruta = BorutaPy(model, n_estimators='auto', verbose=2, random_state=42)
boruta.fit(X_train.values, y_train.values)

# Get the selected features
selected_features = X_train.columns[boruta.support_]
print("Selected features (Boruta):", selected_features)

# Train and evaluate the model
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (Boruta): {mse}")